# Data Ingestion (Alternative)
This is an alternative ingestion method using TIINGO. Note that there are timed limits for free API usage.

### Imports

In [1]:
import requests
import os
from os.path import join, dirname
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import datetime
import json
import psycopg2
import yfinance as yf

### Retrieve env variables

In [ ]:
load_dotenv(find_dotenv())

TIINGO_API_KEY = os.environ.get("TIINGO_API_KEY")
PG_HOST = os.environ.get("PG_HOST")
PG_PORT = os.environ.get("PG_PORT")
PG_DATABASE = os.environ.get("PG_DATABASE")
PG_USER = os.environ.get("PG_USER")
PG_PASSWORD = os.environ.get("PG_PASSWORD")

# TIINGO

In [ ]:
startDate = "2015-01-01"
headers = {
        'Content-Type': 'application/json',
        'Authorization' : f"TOKEN {TIINGO_API_KEY}"
        }

### Testing endpoint and tokens

In [ ]:
requestResponse = requests.get("https://api.tiingo.com/api/test/",
                                    headers=headers)
print(requestResponse.json())

### Retrieve historical daily price information


Get list of updated TIINGO supported tickers: https://apimedia.tiingo.com/docs/tiingo/daily/supported_tickers.zip 

Move the downloaded csv into ./ingestion folder

In [ ]:
tickers_df = pd.read_csv('supported_tickers.csv')
print(tickers_df.head())

   ticker exchange assetType priceCurrency   startDate     endDate
0    -P-H     NYSE     Stock           USD         NaN         NaN
1    -P-S     NYSE     Stock           USD  2018-08-22  2023-05-05
2  000001      SHE     Stock           CNY  2007-01-04  2024-10-23
3  000002      SHE     Stock           CNY  2007-01-04  2024-10-23
4  000003      SHE     Stock           CNY         NaN         NaN


Change into valid dates

In [ ]:
tickers_df['startDate'] = pd.to_datetime(tickers_df['startDate'])
tickers_df['endDate'] = pd.to_datetime(tickers_df['endDate'])

Filter only for current stocks

In [ ]:
tolerance = 365
tickers_df = tickers_df.loc[tickers_df['endDate'] >= (pd.Timestamp.now() - pd.Timedelta(tolerance, unit='D'))]
print(tickers_df.head())

   ticker exchange assetType priceCurrency  startDate    endDate
2  000001      SHE     Stock           CNY 2007-01-04 2024-10-23
3  000002      SHE     Stock           CNY 2007-01-04 2024-10-23
5  000004      SHE     Stock           CNY 2007-08-31 2024-10-23
6  000005      SHE     Stock           CNY 2007-08-31 2024-03-05
7  000006      SHE     Stock           CNY 2007-01-04 2024-10-23


Optional Filter for a single exchange

In [ ]:
exchange = "NYSE"
filtered_tickers_df = tickers_df.loc[tickers_df['exchange'] == exchange]
print(filtered_tickers_df.count())
print(filtered_tickers_df.head())

ticker           3207
exchange         3207
assetType        3207
priceCurrency    3207
startDate        3207
endDate          3207
dtype: int64
      ticker exchange assetType priceCurrency  startDate    endDate
6535       A     NYSE     Stock           USD 1999-11-18 2024-10-23
6548      AA     NYSE     Stock           USD 2016-10-18 2024-10-23
6611     AAC     NYSE     Stock           USD 2021-03-25 2023-11-06
6613   AAC-U     NYSE     Stock           USD 2021-02-02 2023-11-06
6614  AAC-WS     NYSE     Stock           USD 2021-03-25 2023-11-06


Call TIINGO API for historical data

In [ ]:
def tiingo_stock_price(ticker, startDate, headers):
    return requests.get(f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={startDate}", headers=headers)

In [ ]:
for ticker in filtered_tickers_df['ticker']:
    requestResponse = tiingo_stock_price(ticker, startDate, headers)
    # TODO integrate returned data into database as done with yfinance

[{'date': '2015-01-02T00:00:00.000Z', 'close': 12.67, 'high': 12.94, 'low': 12.3898, 'open': 12.94, 'volume': 424724, 'adjClose': 3.4123035638, 'adjHigh': 3.4850203721, 'adjLow': 3.336839676, 'adjOpen': 3.4850203721, 'adjVolume': 849448, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2015-01-05T00:00:00.000Z', 'close': 12.16, 'high': 12.31, 'low': 11.97, 'open': 12.27, 'volume': 458213, 'adjClose': 3.2749495924, 'adjHigh': 3.3153478193, 'adjLow': 3.223778505, 'adjOpen': 3.3045749588, 'adjVolume': 916426, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2015-01-06T00:00:00.000Z', 'close': 12.18, 'high': 12.44, 'low': 11.98, 'open': 12.28, 'volume': 389650, 'adjClose': 3.2803360226, 'adjHigh': 3.3503596159, 'adjLow': 3.2264717201, 'adjOpen': 3.3072681739, 'adjVolume': 779300, 'divCash': 0.0, 'splitFactor': 1.0}, {'date': '2015-01-07T00:00:00.000Z', 'close': 12.95, 'high': 13.01, 'low': 12.3901, 'open': 12.46, 'volume': 446166, 'adjClose': 3.4877135873, 'adjHigh': 3.503872878, 'adjLow': 3.3